In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 20 14:23:40 2017
@author: JTay
"""

import sklearn.model_selection as ms
from sklearn.ensemble import AdaBoostClassifier
#from helper import dtclf_pruned
import helper
from helperimport  basicResults_credit, basicResults_wine, makeTimingCurve,iterationLC_wine, iterationLC_credit
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
# GET THE DATA ============
# wine data
file_path ="./data/"
wine = pd.read_csv (file_path+'wines.csv', sep =",")
# credit data
file_path2 ="./data/"
credit = pd.read_csv (file_path2+'credit.csv', sep =",")

# Break out predicting and target variable data    
wineX = wine.drop('quality',1).copy().values
wineY = wine['quality'].copy().values

creditX = credit.drop('default',1).copy().values
creditY = credit['default'].copy().values



In [ ]:
#check the data
creditX.shape, creditY.shape, wineX.shape, wineY.shape

In [ ]:
# DIVIDE INTO TRAIN AND TEST SETS  
wine_trgX, wine_tstX, wine_trgY, wine_tstY = ms.train_test_split(
    wineX, wineY, test_size=0.3, random_state=0,stratify=wineY)     

credit_trgX, credit_tstX, credit_trgY, credit_tstY = ms.train_test_split(
    creditX, creditY, test_size=0.3, random_state=0,stratify=creditY)   


In [ ]:
# alphas to try

alphas = [-1,-1e-3,-(1e-3)*10**-0.5, -1e-2, -(1e-2)*10**-0.5,-1e-1,-(1e-1)*10**-0.5, 0, (1e-1)*10**-0.5,1e-1,(1e-2)*10**-0.5,1e-2,(1e-3)*10**-0.5,1e-3]


In [ ]:
# define base paramters for base decision tree
wine_base = dtclf_pruned(criterion='gini',class_weight='balanced',random_state=55)                
credit_base = dtclf_pruned(criterion='entropy',class_weight='balanced',random_state=55)
OF_base = dtclf_pruned(criterion='gini',class_weight='balanced',random_state=55)     

In [ ]:
# define parameters

#paramsC= {'Boost__n_estimators':[1,2,5,10,20,30,40,50],'Boost__learning_rate':[(2**x)/100 for x in range(8)]+[1]}
paramsC= {'Boost__n_estimators':[1,2,5,10,20,30,45,60,80,100],
          'Boost__base_estimator__alpha':alphas}
#paramsW = {'Boost__n_estimators':[1,2,5,10,20,30,40,50,60,70,80,90,100],
#           'Boost__learning_rate':[(2**x)/100 for x in range(8)]+[1]}

paramsW = {'Boost__n_estimators':[1,2,5,10,20,30,45,60,80,100],
           'Boost__base_estimator__alpha':alphas}

In [ ]:
# define booster fuctions and associated paramters
wine_booster = AdaBoostClassifier(algorithm='SAMME',learning_rate=1,base_estimator=wine_base,random_state=55)
credit_booster = AdaBoostClassifier(algorithm='SAMME',learning_rate=1,base_estimator=credit_base,random_state=55)
OF_booster = AdaBoostClassifier(algorithm='SAMME',learning_rate=1,base_estimator=OF_base,random_state=55)


In [ ]:
# define the pipelines   
pipeW = Pipeline([('Scale',StandardScaler()),
                  # not doing feature selection
#                  ('Cull1',SelectFromModel(RandomForestClassifier(random_state=1),threshold='median')),
#                  ('Cull2',SelectFromModel(RandomForestClassifier(random_state=2),threshold='median')),
#                  ('Cull3',SelectFromModel(RandomForestClassifier(random_state=3),threshold='median')),
#                  ('Cull4',SelectFromModel(RandomForestClassifier(random_state=4),threshold='median')),
                 ('Boost',wine_booster)])

pipeC = Pipeline([('Scale',StandardScaler()),                
                 ('Boost',credit_booster)])


In [ ]:
# define basic results functions and variables
wine_clf = basicResults_wine(pipeW,wine_trgX,wine_trgY,wine_tstX,wine_tstY,paramsW,'Boost','wine')        
credit_clf = basicResults_credit(pipeC,credit_trgX,credit_trgY,credit_tstX,credit_tstY,paramsC,'Boost','credit')        


In [ ]:
# get the best paramters
wine_final_params = wine_clf.best_params_
credit_final_params = credit_clf.best_params_
print(wine_final_params, credit_final_params)

In [ ]:
#wine_final_params = {'n_estimators': 20, 'learning_rate': 0.02}
#credit_final_params = {'n_estimators': 10, 'learning_rate': 1}
#OF_params = {'learning_rate':1}

# define OF paramters
OF_params = {'Boost__base_estimator__alpha':0.01, 'Boost__n_estimators':50}



In [ ]:
# timing curve - wine
pipeW.set_params(**wine_final_params)
makeTimingCurve(wineX,wineY,pipeW,'Boost','wine')

In [ ]:
# timing curve - credit
pipeC.set_params(**credit_final_params)
makeTimingCurve(creditX,creditY,pipeC,'Boost','credit')

In [ ]:
# learning curve for wine
pipeW.set_params(**wine_final_params)
iterationLC_wine(pipeW,wine_trgX,wine_trgY,wine_tstX,wine_tstY,
    {'Boost__n_estimators':[1,2,5,10,20,30,40,50,60,70,80,90,100]},'Boost','wine')

In [ ]:
# learning curve for credit
pipeC.set_params(**credit_final_params)
iterationLC_credit(pipeC,credit_trgX,credit_trgY,credit_tstX,credit_tstY,
    {'Boost__n_estimators':[1,2,5,10,20,30,40,50]},'Boost','credit')                

In [ ]:
# learning boost OF - wine
pipeW.set_params(**OF_params)
iterationLC_wine(pipeW,wine_trgX,wine_trgY,wine_tstX,wine_tstY,
    {'Boost__n_estimators':[1,2,5,10,20,30,40,50,60,70,80,90,100]},'Boost_OF','wine')                

In [ ]:
# learning curve boost OF - credit
pipeC.set_params(**OF_params)
iterationLC_credit(pipeC,credit_trgX,credit_trgY,credit_tstX,credit_tstY,
    {'Boost__n_estimators':[1,2,5,10,20,30,40,50]},'Boost_OF','credit')                
